### importing the python libraries

In [1]:
import pandas as  pd # Pandas libraries for manipulation of data
import sqlite3 # Database libraries

### initialization

In [2]:
conn = sqlite3.connect("data.db") # Creating data base called data. 
curr = conn.cursor() # Connecting the created database to conn.cursor() method.

### Importing the csv files

### Features_data - Table

In [3]:
df = pd.read_csv('Features_data_set.csv') # Importing the Features data set

In [4]:
df.shape # Finding the number of observations and variables in the data set.

(8190, 12)

In [5]:
df.ndim # Finding the number of dimensions

2

In [6]:
df.info() # Understanding the structure of the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [7]:
df.describe() # Statistical description of the dataset.

,Store,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
count,8190.000000,8190.000000,8190.000000,4032.000000,2921.000000,3613.000000,3464.000000,4050.000000,7605.000000,7605.000000
mean,23.000000,59.356198,3.405992,7032.371786,3384.176594,1760.100180,3292.935886,4132.216422,172.460809,7.826821
std,12.987966,18.678607,0.431337,9262.747448,8793.583016,11276.462208,6792.329861,13086.690278,39.738346,1.877259
min,1.000000,-7.290000,2.472000,-2781.450000,-265.760000,-179.260000,0.220000,-185.170000,126.064000,3.684000
25%,12.000000,45.902500,3.041000,1577.532500,68.880000,6.600000,304.687500,1440.827500,132.364839,6.634000
50%,23.000000,60.710000,3.513000,4743.580000,364.570000,36.260000,1176.425000,2727.135000,182.764003,7.806000
75%,34.000000,73.880000,3.743000,8923.310000,2153.350000,163.150000,3310.007500,4832.555000,213.932412,8.567000
max,45.000000,101.950000,4.468000,103184.980000,104519.540000,149483.310000,67474.850000,771448.100000,228.976456,14.313000


#### Data Preprocessing on Features_data table

In [8]:
df.isna().sum() # Finding the missing values in the variables.

Store              0
Date               0
Temperature        0
Fuel_Price         0
MarkDown1       4158
MarkDown2       5269
MarkDown3       4577
MarkDown4       4726
MarkDown5       4140
CPI              585
Unemployment     585
IsHoliday          0
dtype: int64

In [9]:
# replacing NaN in the MarkDown variables with 0.
df['MarkDown1'] = df['MarkDown1'].fillna(0)
df['MarkDown2'] = df['MarkDown2'].fillna(0)
df['MarkDown3'] = df['MarkDown3'].fillna(0)
df['MarkDown4'] = df['MarkDown4'].fillna(0)
df['MarkDown5'] = df['MarkDown5'].fillna(0)

In [10]:
# Converting negative values to positive values using abs() function
df['MarkDown1'] = df['MarkDown1'].abs()
df['MarkDown2'] = df['MarkDown2'].abs()
df['MarkDown3'] = df['MarkDown3'].abs()
df['MarkDown4'] = df['MarkDown4'].abs()
df['MarkDown5'] = df['MarkDown5'].abs()
df['Temperature'] = df['Temperature'].abs()
df['Fuel_Price'] = df['Fuel_Price'].abs()
df['CPI'] = df['CPI'].abs()
df['Unemployment'] = df['Unemployment'].abs()



In [11]:
# Imputation on CPI and Unemployment variables with median 
# df.fillna(df.median(numeric_only=True).round(1),inplace=True )
df.fillna({"CPI": df['CPI'].median(),"Unemployment": df['Unemployment'].median()}, inplace=True)


In [12]:
# Confirming the cleaning of the dataset.
df.isnull().sum()

Store           0
Date            0
Temperature     0
Fuel_Price      0
MarkDown1       0
MarkDown2       0
MarkDown3       0
MarkDown4       0
MarkDown5       0
CPI             0
Unemployment    0
IsHoliday       0
dtype: int64

In [13]:
df.head(5) # View the first five observations of the dataset

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,05/02/2010,42.31,2.572,0.0,0.0,0.0,0.0,0.0,211.096358,8.106,False
1,1,12/02/2010,38.51,2.548,0.0,0.0,0.0,0.0,0.0,211.242170,8.106,True
2,1,19/02/2010,39.93,2.514,0.0,0.0,0.0,0.0,0.0,211.289143,8.106,False
3,1,26/02/2010,46.63,2.561,0.0,0.0,0.0,0.0,0.0,211.319643,8.106,False
4,1,05/03/2010,46.50,2.625,0.0,0.0,0.0,0.0,0.0,211.350143,8.106,False


#### Creating columns and inserting the features table into the database - data in SQLite

In [14]:
curr.execute("drop table if exists features_data") 
# creating the variables or columns for the features table
curr.execute("create table features_data (store int,date Text, temperature double, fuel_price double,mk1 double,mk2 double,mk3 double,mk4 double,mk5 double,cpi double,unemployment double,FOREIGN KEY (date) REFERENCES date_isholiday (date),FOREIGN KEY (store) REFERENCES store_data (store),PRIMARY KEY (store,date), unique(store,date))")
print("table created successfully")

table created successfully


In [15]:
# df2 = df2.reset_index()   # make sure indexes pair with number of rows
for index,row in df.iterrows():
    curr.execute("insert into features_data (store,date, temperature, fuel_price, mk1,mk2,mk3,mk4,mk5, cpi,unemployment) values(?,?,?,?,?,?,?,?,?,?,?)",(row['Store'],row['Date'],row['Temperature'],row['Fuel_Price'],row['MarkDown1'],row['MarkDown2'],row['MarkDown3'],row['MarkDown4'],row['MarkDown5'],row['CPI'],row['Unemployment']))
conn.commit() # Inserting and saving the data into the database
print("data inserted successfullly")

data inserted successfullly


### Date_holiday- Table

In [16]:
date_isholiday = {
    "date":df['Date'].tolist(),
    "isholiday":df['IsHoliday'].tolist()
}

ddh = pd.DataFrame(date_isholiday) #Converting the date to datafame

In [17]:
ddh =ddh.drop_duplicates(subset='date', keep="first")

#### Creating columns and inserting the Date_holiday table into the database - data in SQLite

In [18]:
curr.execute("drop table if exists date_isholiday")
# creating the variables or columns for the Date_holiday table
curr.execute("create table date_isholiday (date Text,isholiday BOOLEAN,PRIMARY KEY (date),unique(date))")
print("table created successfully")

table created successfully


In [19]:
# df2 = df2.reset_index()   # make sure indexes pair with number of rows
for index,row in ddh.iterrows():
    curr.execute("insert into date_isholiday (date,isholiday) values(?,?)",(row['date'],row['isholiday']))
conn.commit() # Inserting and saving the data into the database
print("data inserted successfullly")

data inserted successfullly


### Manager - Table

In [20]:
df1 = pd.read_csv('store_info.csv') # Loading the store_info table

In [21]:
df1.shape # Finding out the number of observations and varibles/columns that make up the dataset

(48, 5)

In [22]:
df1.ndim # The dimension of the dataset.

2

In [23]:
df1.info() # Understanding the structure of the dataset and the variables datatypes.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Store             48 non-null     int64 
 1   Manager           45 non-null     object
 2   Years_as_manager  48 non-null     int64 
 3   Email             46 non-null     object
 4   Address           48 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.0+ KB


In [24]:
df1.describe() # Statistical description of the numerical variables.

,Store,Years_as_manager
count,48.00,48.000000
mean,24.50,10.020833
std,14.00,14.052318
min,1.00,0.000000
25%,12.75,3.750000
50%,24.50,9.000000
75%,36.25,12.250000
max,48.00,100.000000


#### Data Pre-processing - Store_info

In [25]:
df1.isna().sum() # Finding the variables with null values

Store               0
Manager             3
Years_as_manager    0
Email               2
Address             0
dtype: int64

In [26]:
# df1 = df1[df1['Manager'].notna()]
# replacement of empty cells with values
df1["Manager"] = df1["Manager"].fillna("noname noname")
df1["Email"] = df1["Email"].fillna("no email")

In [27]:
df1.isna().sum() # Confirming the replacement of the empty cells

Store               0
Manager             0
Years_as_manager    0
Email               0
Address             0
dtype: int64

In [28]:
# Splitting of the manager's name into first and last names.
firstName = []
lastName = []
idd = []

d = df1['Manager'].tolist()

for idx, val in enumerate(d):
    newVal = val.split(" ")
    firstName.append(newVal[0])
    lastName.append(newVal[1])
    dd = idx+1
    idd.append(dd)
    

    
df1["first_name"] = firstName
df1["last_name"] = lastName
df1["manager_id"] = idd



 
    
    

In [29]:
del df1["Manager"] # getting rid of the manager's variable


In [30]:
df1.head(5) # View the first five observations after the splitting of names.

,Store,Years_as_manager,Email,Address,first_name,last_name,manager_id
0,1,1,Robert.Alvey@Walmart.org,4971 Janet Court;Livermore;CA;94550,Robert,Alvey,1
1,2,11,Jerry.Martinez@Walmart.org,4439 Gale Street;Livermore;CA;94550,Jerry,Martinez,2
2,3,7,Susanna.Kellner@Walmart.org,856 Milton Street;Oakland;CA;94607,Susanna,Kellner,3
3,4,1,Marco.Spivey@Walmart.org,474 Merritt Avenue;Oakland;CA;94610,Marco,Spivey,4
4,5,13,Timothy.Narvaez@Walmart.org,129 Grant Street;Redlands;CA;92373,Timothy,Narvaez,5


#### Creating columns and inserting the Store_info table into the database - data in SQLite

In [31]:
curr.execute("drop table if exists manager")
# creating the variables or columns for the Store_info table
curr.execute("create table manager (manager_id text,first_name Text,last_name Text, year_as_manager int, email Text,postal_code int,PRIMARY KEY (manager_id),FOREIGN KEY (postal_code) REFERENCES address (postal_code),unique(manager_id))")
print("table created successfully")


table created successfully


In [32]:
## Transformation on the Address
data = df1["Address"].tolist()
city = []
state = []
postal =[]
address = []
for item in data:
    newData=item.split(";")
    city.append(newData[1])
    state.append(newData[2])
    postal.append(newData[3])
    address.append(newData[0])

In [33]:
del df1['Address']

In [34]:
df1["postal_code"] = postal

In [35]:
# df2 = df2.reset_index()   # make sure indexes pair with number of rows
for index,row in df1.iterrows():
    curr.execute("insert into manager (manager_id,first_name,last_name,year_as_manager,email,postal_code) values(?,?,?,?,?,?)",(row['manager_id'],row['first_name'],row['last_name'],row['Years_as_manager'],row['Email'],row['postal_code']))
conn.commit() # Inserting and saving the data into the database
print("data inserted successfullly")

data inserted successfullly


In [36]:
df1.head() # First five observations on the table.

,Store,Years_as_manager,Email,first_name,last_name,manager_id,postal_code
0,1,1,Robert.Alvey@Walmart.org,Robert,Alvey,1,94550
1,2,11,Jerry.Martinez@Walmart.org,Jerry,Martinez,2,94550
2,3,7,Susanna.Kellner@Walmart.org,Susanna,Kellner,3,94607
3,4,1,Marco.Spivey@Walmart.org,Marco,Spivey,4,94610
4,5,13,Timothy.Narvaez@Walmart.org,Timothy,Narvaez,5,92373


### Address- table

In [37]:
# Application of dictionary to the address
address = {
    "postal_code":df1["postal_code"].tolist(),
    "city":city,
    "state":state,
    "address":address
}



daddr = pd.DataFrame(address)

In [38]:
daddr.head() # First five observation of the Address table

,postal_code,city,state,address
0,94550,Livermore,CA,4971 Janet Court
1,94550,Livermore,CA,4439 Gale Street
2,94607,Oakland,CA,856 Milton Street
3,94610,Oakland,CA,474 Merritt Avenue
4,92373,Redlands,CA,129 Grant Street


In [39]:
daddr = daddr.drop_duplicates(subset='postal_code', keep="first")

#### Creating columns and inserting the Address table into the database - data in SQLite¶

In [40]:
curr.execute("drop table if exists address")
# creating the variables or columns for the Address table
curr.execute("create table address (postal_code int,city Text, state Text,address Text, PRIMARY KEY (postal_code), unique(postal_code))")
print("table created successfully")

table created successfully


In [41]:
# df2 = df2.reset_index()   # make sure indexes pair with number of rows
for index,row in daddr.iterrows():
    curr.execute("insert into address (postal_code,city,state,address) values(?,?,?,?)",(row['postal_code'],row['city'],row['state'],row['address'],))
conn.commit() # Inserting and saving the data into the database
print("data inserted successfullly")

data inserted successfullly


### Manager_store- Table

In [42]:
manager_store = {
    "store":df1['Store'].tolist(),
    "manager_id":df1['manager_id'].tolist()
}

dms = pd.DataFrame(manager_store)

In [43]:
dms.head(9)

,store,manager_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5
5,6,6
6,7,7
7,8,8
8,9,9


#### Creating columns and inserting the Manager_store  table into the database - data in SQLite¶

In [44]:
curr.execute("drop table if exists manager_store")
# creating the variables or columns for the Manager_store table
curr.execute("create table manager_store (store Int,manager_id Text, PRIMARY KEY (store), FOREIGN KEY (store) REFERENCES store_data (store),FOREIGN KEY (manager_id) REFERENCES manager (manager_id),unique(store))")
print("table created successfully")

table created successfully


In [45]:
# df2 = df2.reset_index()   # make sure indexes pair with number of rows
for index,row in dms.iterrows():
    curr.execute("insert into manager_store (manager_id,store) values(?,?)",(row['manager_id'],row['store']))
conn.commit() # Inserting and saving the data into the database
print("data inserted successfullly")

data inserted successfullly


### Stores Data - Table

In [46]:
df2 = pd.read_csv('stores_data-set.csv') # Loading the stores_data-set

In [47]:
df2.shape # Finding the number of observations and variables

(45, 3)

In [48]:
df2.ndim # The dimension of the dataset

2

In [49]:
df2.info() # Understanding the structure of the dataset and data types of the variables.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Store   45 non-null     int64  
 1   Type    45 non-null     object 
 2   Size    43 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.2+ KB


In [50]:
df2.describe() # The statistical description of the numerical variables

,Store,Size
count,45.000000,43.000000
mean,23.000000,129923.069767
std,13.133926,65228.295508
min,1.000000,34875.000000
25%,12.000000,63955.000000
50%,23.000000,126512.000000
75%,34.000000,202406.000000
max,45.000000,219622.000000


#### Data Pre-processing

In [51]:
df2.isna().sum() # Checking for null values

Store    0
Type     0
Size     2
dtype: int64

In [52]:
df2.fillna({'Size':df2['Size'].median()}, inplace=True) # Imputation with median technique.

In [53]:
df2.isna().sum() # Viewing after data cleaning.

Store    0
Type     0
Size     0
dtype: int64

#### Creating columns and inserting the Store_data-set  table into the database - data in SQLite¶

In [54]:
curr.execute("drop table if exists store_data")
# creating the variables or columns for the store_data table
curr.execute("create table store_data (store int primary key unique,  type text, size int)")
print("table created successfully")

table created successfully


In [55]:
# df2 = df2.reset_index()   # make sure indexes pair with number of rows
for index,row in df2.iterrows():
    curr.execute("insert into store_data (store,type,size) values(?,?,?)",(row['Store'],row['Type'],row['Size']))
    conn.commit()  # Inserting and saving the data into the database
print("data inserted successfullly")

data inserted successfullly


### sales_data - Table

In [56]:
df3 = pd.read_csv('sales_data-set.csv') # Loading the sales_data-set dataset

In [57]:
df3.shape # Finding the number of observations and variables

(421570, 5)

In [58]:
df3.ndim # The dimension of the dataset

2

In [59]:
df3.info() # Understanding the data structure and the data type of the variables

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [60]:
df3.describe() # Statistical description of the dataset.

,Store,Dept,Weekly_Sales
count,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,15981.258123
std,12.785297,30.492054,22711.183519
min,1.000000,1.000000,-4988.940000
25%,11.000000,18.000000,2079.650000
50%,22.000000,37.000000,7612.030000
75%,33.000000,74.000000,20205.852500
max,45.000000,99.000000,693099.360000


#### Data Pre-processing 

In [61]:
df3.isna().sum() # Checking for null values

Store           0
Dept            0
Date            0
Weekly_Sales    0
IsHoliday       0
dtype: int64

#### Creating columns and inserting the Sales_data-set table into the database - data in SQLite¶

In [62]:
curr.execute("drop table if exists sales_data")
# creating the variables or columns for the sales_data table
curr.execute("create table sales_data (store int,department int, date Text, weekly_sales double, FOREIGN KEY (date) REFERENCES date_isholiday (date),FOREIGN KEY (store) REFERENCES store_data (store),PRIMARY KEY (store,department, date), unique(store,department, date))")
print("table created successfully")

table created successfully


In [63]:
# df2 = df2.reset_index()   # make sure indexes pair with number of rows
for index,row in df3.iterrows():
    curr.execute("insert into sales_data (store,department,date,weekly_sales) values(?,?,?,?)",(row['Store'],row['Dept'],row['Date'],row['Weekly_Sales']))
conn.commit()  # Inserting and saving the data into the database
print("data inserted successfullly")

data inserted successfullly
